<a href="https://colab.research.google.com/github/IwinosaA/Assignment/blob/main/Iwinosa_A_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Default title text
'''
Name: Iwinosa Agbonlahor
Class: EEGR 565 Machine Learning Applications
Assignment 2:
This is a Program that reads the 'message' in a csv file to determine if it is a spam or not
An unsupervised model K Means and a supervised Multinomial naive Bayes model was implemented

Link to shared folder: https://drive.google.com/drive/folders/109C0OOSGMVeoHsjont0MkVvMQdvrNNCK?usp=sharing
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from nltk.corpus import names
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

In [ ]:
import chardet
with open('/content/drive/My Drive/Iwinosa_Agbonlahor_EEGR 565_Fall2022/spam.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df = pd.read_csv('/content/drive/My Drive/Iwinosa_Agbonlahor_EEGR 565_Fall2022/spam.csv', encoding = 'Windows-1252')
df.head()

,label,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.message

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568              Will ï¿½_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [ ]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords


      Unzipping corpora/stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> names


      Unzipping corpora/names.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet2021



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
def text_clean(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
df.message.head(5).apply(text_clean)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [ ]:
X = df.message.apply(text_clean)
y = df.label
#X
#y

In [ ]:
#Functions to Clean the data
all_names = set(x.lower() for x in names.words())
lemmatizer = WordNetLemmatizer()

# Function that checks if a letter does not have numbers
def letters_only(astr):
    return astr.isalpha()

stops = stopwords.words("english")
def clean(words, stops):
  return[lemmatizer.lemmatize(word) for word in words if (word not in stops) and word not in all_names and letters_only(word)]

In [ ]:
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Remove names from words and perform word lemmatization
zag = [" ".join(clean(tokens, stops)) for tokens in X]
tokens = [x.lower() for x in zag]

In [ ]:
tokens

['go jurong point crazy available bugis n great world e buffet cine got amore',
 'ok lar joking wif u oni',
 'free entry wkly comp fa cup final tkts may text fa receive entry questionstd txt ratetcs apply',
 'u dun early hor u c already',
 'nah dont think go usf life around though',
 'freemsg hey darling week word back id like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune callers press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'im gonna home soon dont want talk stuff anymore tonight k ive cried enough today',
 'six chance cash pound txt send cost tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim tc wwwdbuknet lccltd pobox',
 'ive searching right word th

In [ ]:
# Making a DataFrame of the clean data
data = pd.DataFrame({
    'Message': tokens,
    'Label': y
})
data.head()

,Message,Label
0,go jurong point crazy available bugis n great ...,ham
1,ok lar joking wif u oni,ham
2,free entry wkly comp fa cup final tkts may tex...,spam
3,u dun early hor u c already,ham
4,nah dont think go usf life around though,ham


In [ ]:
# Encoding the labels
encoder = preprocessing.LabelEncoder()
y_data = encoder.fit_transform(y)

# Vectorizing the features using TFID Vectorizer
vectorizer = TfidfVectorizer(max_features=5000,)
X_data= vectorizer.fit_transform(tokens)

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,train_size=0.7,random_state=42)

# Using Kmeans to cluster into 2 Clusters
kmeans = KMeans(n_clusters=2, random_state=42).fit(X_train)

# Making predictions on the test data
y_pred =  kmeans.predict(X_test)

from sklearn import metrics
print('Accuracy:',metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8827751196172249


In [ ]:
X_array = X_train.toarray()

# Converting the array into a dataframe 
data_df = pd.DataFrame(X_array, columns = vectorizer.get_feature_names_out())
data_df.head()

,aa,aah,aathilove,aathiwhere,abi,ability,abiola,abj,able,abt,...,zac,zahers,zealand,zebra,zero,zhong,zindgi,zoe,zogtorius,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data_df['labels'] = data.Label

# Getting the top 25 spam vectorized tokens
spam = data_df[data_df['labels'] == 'spam']
spamx = spam.max()[0:5000].reset_index()
spamx.rename(columns = {'index':'Words', 0:'TF_IDF'}, inplace = True)
spam_data = spamx.sort_values(by=['TF_IDF'], ascending=False)
spam_data.head(25)

,Words,TF_IDF
638,creep,1.0
2313,player,1.0
2110,networking,1.0
3820,talk,1.0
1397,happened,1.0
1395,happen,1.0
488,check,1.0
2137,nope,1.0
665,da,1.0
3981,thinking,1.0


In [ ]:
# Getting the top 25 of the ham vectorized tokens
ham = data_df[data_df['labels'] == 'ham']
hamx = ham.max()[0:5000].reset_index()
hamx.rename(columns = {'index':'Words', 0:'TF_IDF'}, inplace = True)
ham_data = hamx.sort_values(by=['TF_IDF'], ascending=False)
ham_data.head(25)

,Words,TF_IDF
2126,nite,1.0
4795,wot,1.0
3824,talking,1.0
3820,talk,1.0
1946,meeting,1.0
687,days,1.0
1461,holding,1.0
1467,home,1.0
2408,practising,1.0
2665,real,1.0


In [ ]:
# SUPERVISED LEARNING USING MULTINOMIAL NAIVE BAYES

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=1, fit_prior=True)
mnb.fit(X_train, y_train)

accuracy = mnb.score(X_test, y_test)
print("Accuracy from MultinomialNB = ",accuracy)

Accuracy from MultinomialNB =  0.9641148325358851


Its such a fulfilment to achieve this much.
I hope next time I enjoy the extra Credit.
Thank You Prof.